In [ ]:
# Gerekli kütüphaneleri yükleyelim
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Model için Sequential sınıfı
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # CNN için gerekli katmanlar
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image  # Görüntü işlemleri için PIL kütüphanesi

# 1. Veri setini yükleme ve eğitim/test setlerini ayırma
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# MNIST veri seti, 60,000 eğitim örneği ve 10,000 test örneği içerir.
# X_train, 28x28 boyutunda el yazısı rakamları içeren görüntülerdir.
# y_train, bu görüntülere karşılık gelen etiketlerdir (0-9 arası rakamlar).
# X_test ve y_test ise test verilerini içerir.

# 2. Veri ön işleme (normalizasyon ve yeniden şekillendirme)
X_train = X_train / 255.0  # Görüntülerin piksel değerlerini 0-255 aralığından 0-1 aralığına dönüştürür.
X_test = X_test / 255.0    # Aynı işlemi test verisi için yapıyoruz.

# Görüntüleri modelin kabul edeceği şekle getirmek: (num_samples, height, width, channels)
# Burada her görüntü 28x28 boyutunda olduğu için, her birini tek kanal (siyah-beyaz) olarak şekillendiriyoruz.
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# 3. Modeli tanımlama
model = Sequential([
    # İlk konvolüsyonel katman: 32 adet 3x3 filtre kullanarak, aktivasyon fonksiyonu olarak ReLU kullanıyoruz.
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),  # Maksimum havuzlama: 2x2 havuzlama alanı
    Dropout(0.25),  # Dropout katmanı: Aşırı öğrenmeyi engellemek için rastgele bağlantıları devre dışı bırakır

    # İkinci konvolüsyonel katman: 64 adet 3x3 filtre
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),  # Maksimum havuzlama: 2x2 havuzlama alanı
    Dropout(0.25),

    Flatten(),  # Konvolüsyonel katmanlardan çıkan özellik haritalarını düzleştirir (1D vektör haline getirir)
    Dense(128, activation='relu'),  # Tam bağlantılı katman: 128 nöron, ReLU aktivasyon fonksiyonu
    Dropout(0.5),  # Dropout: Bu katmanda da aşırı öğrenmeyi engellemeye çalışıyoruz
    Dense(10, activation='softmax')  # Son katman: 10 sınıf için softmax aktivasyonu (sınıflandırma için)
])

# 4. Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Derleme işleminde:
# - Optimizer: Adam (öğrenme oranını uyarlayan bir optimizasyon algoritması)
# - Loss: Sparse Categorical Crossentropy (çok sınıflı sınıflandırma problemi için uygun kayıp fonksiyonu)
# - Metrics: Doğruluk (accuracy), modelin doğruluğunu değerlendireceğiz

# 5. Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

# model.fit:
# - X_train, y_train: Eğitim verisi ve etiketleri
# - epochs: Modelin eğitim verisi üzerinde kaç kez geçeceğini belirtir (10 epok)
# - validation_data: Modelin her epoch sonunda doğrulama (test) verisi üzerinde nasıl performans gösterdiğini görmemizi sağlar.
# - batch_size: Eğitim sırasında her seferinde kaç örneğin işleneceğini belirler (64)

# 6. Model performansını değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Modeli test verisiyle değerlendiririz.
# test_loss: Test setindeki kayıp değeri
# test_accuracy: Test setindeki doğruluk oranı

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# 7. Yeni bir görüntüyü yükleyip sınıflandırma

# Yeni bir görüntü yükleyelim
img = Image.open('yüklenecek görüntü.jpg').convert('L')  # Siyah-beyaz bir resim açıyoruz
img = img.resize((28, 28))  # Görüntüyü 28x28 boyutlarına getiriyoruz
img_array = np.array(img) / 255.0  # Görüntüyü normalize ediyoruz
img_array = img_array.reshape(1, 28, 28, 1)  # Modelin beklediği şekilde şekillendiriyoruz

# 8. Görüntü üzerinde tahmin yapma
prediction = model.predict(img_array)

# model.predict: Yeni bir görüntü üzerinde tahmin yapar
# prediction, 10 sınıf için modelin tahmin ettiği olasılıkları döndürür.

# Tahmin edilen sınıf
predicted_label = np.argmax(prediction)  # En yüksek olasılıkla tahmin edilen sınıf

print(f"Tahmin edilen sınıf: {predicted_label}")
